---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [9]:
spam_data.target.value_counts(normalize=True)[1] * 100

13.406317300789663

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [10]:
def answer_one():
    
    return spam_data.target.value_counts(normalize=True)[1] * 100

In [11]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    vect = CountVectorizer().fit(X_train)
    tokens = vect.get_feature_names()
    longest_word = max(tokens, key= lambda w: len(w))
    
    return longest_word

In [13]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    pred = model.predict(vect.transform(X_test))
    
    result = roc_auc_score(y_test, pred)
    return result

In [16]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [167]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    ft = pd.DataFrame(X_train_vectorized.toarray(), columns=vect.get_feature_names()) 
    ft1 = ft[ft > 0]
    
    return (ft1.max(0).nsmallest(20), ft.max(0).nlargest(20))

In [168]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 afternoons      0.091250
 approaching     0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [47]:
def answer_five():
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)

    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    pred = model.predict(vect.transform(X_test))
    
    result = roc_auc_score(y_test, pred)
    
    return result

In [48]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [69]:
def answer_six():
    
    not_spam = spam_data[spam_data['target'] == 0]
    spam = spam_data[spam_data['target'] == 1]
    
    not_spam_length = not_spam.text.apply(len).mean()
    spam_length = spam.text.apply(len).mean()
    
    return not_spam_length, spam_length

In [70]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [71]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [77]:
from sklearn.svm import SVC

def answer_seven():
    
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_add = add_feature(X_train_vectorized, X_train.apply(len))
    
    model = SVC(C=10000)
    model.fit(X_train_add, y_train)
    
    X_test_add = add_feature(vect.transform(X_test), X_test.apply(len))
    pred = model.predict(X_test_add)
    
    result = roc_auc_score(y_test, pred)
    
    return result

In [78]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [98]:
import re
def answer_eight():
    
    not_spam = spam_data[spam_data['target'] == 0]
    spam = spam_data[spam_data['target'] == 1] 
    
    dig1 = [len(re.findall(r'[0-9]', s)) for s in not_spam.text]
    ave1 = sum(dig1) / len(dig1)
    
    dig2 = [len(re.findall(r'[0-9]', s)) for s in spam.text]
    ave2 = sum(dig2) / len(dig2)
    
    return (ave1, ave2)

In [99]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [104]:
from sklearn.linear_model import LogisticRegression

def digits_find(string):
    return len(re.findall(r'[0-9]', string))
    
def answer_nine():
    
    vect = TfidfVectorizer(min_df=5, ngram_range=(1, 3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_add = add_feature(X_train_vectorized, X_train.apply(len))
    X_train_add1 = add_feature(X_train_add, X_train.apply(digits_find))
    
    model = LogisticRegression(C=100)
    model.fit(X_train_add1, y_train)

    X_test_add = add_feature(vect.transform(X_test), X_test.apply(len))
    X_test_add1 = add_feature(X_test_add, X_test.apply(digits_find))
    
    pred = model.predict(X_test_add1)

    return roc_auc_score(y_test, pred)

In [105]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [106]:
def non_word(string):
    return len(re.findall(r'\W', string))

def answer_ten():
    
    not_spam = spam_data[spam_data['target'] == 0]
    spam = spam_data[spam_data['target'] == 1] 
    
    dig1 = not_spam.text.apply(non_word)
    ave1 = sum(dig1) / len(dig1)
    
    dig2 = spam.text.apply(non_word)
    ave2 = sum(dig2) / len(dig2)
    
    return (ave1, ave2)

In [107]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [153]:
def non_word(string):
    return len(re.findall(r'\W', string))

def digits_find(string):
    return len(re.findall(r'[0-9]', string))

def answer_eleven():
    
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_add = add_feature(X_train_vectorized, X_train.apply(len))
    X_train_add1 = add_feature(X_train_add, X_train.apply(digits_find))
    X_train_add2 = add_feature(X_train_add1, X_train.apply(non_word)) 
    
    
    model = LogisticRegression(C=100)
    model.fit(X_train_add2, y_train)

    X_test_add = add_feature(vect.transform(X_test), X_test.apply(len))
    X_test_add1 = add_feature(X_test_add, X_test.apply(digits_find))
    X_test_add2 = add_feature(X_test_add1, X_test.apply(non_word))   
    
    pred = model.predict(X_test_add2)
    result = roc_auc_score(y_test, pred)
    
    return (result, np.sort(model.coef_[0][:10]), np.sort(model.coef_[0])[::-1][:10])

In [154]:
answer_eleven()

(0.97885931107074342,
 array([ -1.45706306e-01,  -1.32252459e-01,  -1.03557321e-01,
         -2.11397607e-02,  -7.97121935e-04,  -2.04055586e-04,
         -1.92079510e-04,  -1.39390180e-05,  -1.39390180e-05,
          2.29209810e-02]),
 array([ 1.21222067,  0.5977756 ,  0.54148602,  0.5387577 ,  0.52147824,
         0.52035676,  0.51786701,  0.51609223,  0.50866479,  0.50264898]))